In [ ]:
import time
import logging
import os
import json
import datetime as dt 
from os.path import join
from logging.handlers import RotatingFileHandler
import pandas as pd
import psycopg2
import ibm_db
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta
import requests

In [ ]:
#1
json_path = r'C:\Users\00220401626\Desktop\teams\config.json'
teams_webhook_url = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/c008d92acff64b93a35fa912edecdbf3/329f5970-aad5-4b39-90b5-cf6569c670e1"             

try:
    with open(json_path) as json_file:
        config = json.load(json_file)

    # Extract Configuration values
    database_1 = config['database_1']
    host_1 = config['host_1']
    port_1 = config['port_1']
    protocol_1 = config['protocol_1']
    user_1 = config['user_1']
    password_1 = config['password_1']
    
    database_2 = config['database_2']
    host_2 = config['host_2']
    port_2 = config['port_2']
    protocol_2 = config['protocol_2']
    user_2 = config['user_2']
    password_2 = config['password_2']

    log = config['log']  
    days = int(config['days'])
    insertion_time = str(config['insertion_time'])
    inserted_time = dt.datetime.strptime(insertion_time, '%Y-%m-%d %H:%M:%S')
    inserted_day = inserted_time.strftime('%Y%m%d')

    # Logging Settings    
    if not os.path.exists(log):
        os.makedirs(log, exist_ok=True)
        
    dt_st = datetime.now()
    log_path = join(log, f"debug_{dt_st.strftime('%Y%m%d')}.log")
    log_formatter = logging.Formatter('%(asctime)s [%(levelname)s]: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    log_handler = RotatingFileHandler(filename=log_path, maxBytes=1048576, backupCount=10, delay=True)
    log_handler.setFormatter(log_formatter)
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    logger.addHandler(log_handler)

    logging.info("Step_1: Extract JSON Configuration Values")
    print("Step_1: Extract JSON Configuration Values")

except Exception as e:
    logging.error(f"Step_1: An error occurred while loading configuration: {e}")
    print(f"Step_1: An error occurred while loading configuration: {e}")

In [ ]:
import psycopg2
import pandas as pd
import logging

def take_data_from_database():
    try:
        conn = psycopg2.connect(database=database_1, user=user_1, password=password_1, host=host_1, port=port_1)

        sql_query = """
            SELECT *
            FROM mm_01 
            WHERE state IN (2, 3) AND timedate > %s
            UNION ALL
            SELECT *
            FROM mm_02 
            WHERE state IN (2, 3) AND timedate > %s;
        """
        
        # Execute SQL query with parameters
        df = pd.read_sql_query(sql_query, conn, params=(inserted_time, inserted_time))

        # Close the database connection
        conn.close()

        # Log and return the DataFrame
        logging.info("Step_2: Successfully fetched data from the database")
        return df
    
    except Exception as e:
        # Log and print any errors
        logging.error(f"Step_2: Error while connecting to PostgreSQL: {e}")
        print(f"Step_2: Error while connecting to PostgreSQL: {e}")

df = take_data_from_database()

In [ ]:
#2
def take_data_from_database():  
    try:
        conn = psycopg2.connect(database=database_1, user=user_1, password=password_1, host=host_1, port=port_1)
        sql_query = """
                    SELECT *
                    FROM mm_01
                    WHERE state IN (2, 3) AND timedate > %s;
                """
        
        df = pd.read_sql_query(sql_query, conn, params=(inserted_time,))

        logging.info("Step_2: Extract JSON Configuration Values")
        print("Step_2: Extract JSON Configuration Values")

        return df
    
    except Exception as e:
        logging.error(f"Step_2: Error while connecting to PostgreSQL: {e}")
        print(f"Step_2: Error while connecting to PostgreSQL: {e}")
        
df=take_data_from_database()
df

In [ ]:
#3 
def insert_into_database(df):
    try:
        dsn = f"DATABASE={database_2};HOSTNAME={host_2};PORT={port_2};PROTOCOL={protocol_2};UID={user_2};PWD={password_2}"
        conn = ibm_db.connect(dsn, "", "")
        if conn:
            logging.info("Step_3: 1.Connected to Database")
            print("Step_3: 1.Connected to Database")
    except Exception as e:
        print("Step_3: 1.Failed to Connect to Database:", e)
        logging.error("Step_3: 1.Failed to Connect to Database:", e)


    values = ''
    for index, row in df.iterrows():
        value_1 = 'Particles'
        value_2 = 'MM_01'
        value_3 = row['timedate']
        value_4 = row['state']
        value_5 = '0.5, 1.0, 5.0, 10.0'
        value_6 = f"{row['0.5']}, {row['1.0']}, {row['5.0']}, {row['10.0']}"

        values += f"('{value_1}','{value_2}', '{value_3}', '{value_4}', '{value_5}', '{value_6}'),"
        
    values = values[:-1] 

    insert_query = f"INSERT INTO IOT_DATA.THRESHOLD_ALERT(CATEGORY, EQP, TS_DT, STATE, DATA_COLUMN, DATA_VALUE) VALUES {values};"
    stmt = ibm_db.prepare(conn, insert_query)

    try:
        if ibm_db.execute(stmt):
            logging.info("Step_3: 2.Inserted to Database")
            print("Step_3: 2.Inserted to Database")

    except Exception as e:
        logging.error(f"Step_3: 2.Error: {e}")
        print(f"Step_3: 2.Error: {e}")

insert_into_database(df)

In [ ]:
#4
def send_teams_message(df):
    message_text = ""

    for index, row in df.iterrows():
        TITLE = row['timedate']
        value_1 = row['state']  
        value_2 = 'MM_01'  
        value_3 = row['0.5']  
        value_4 = row['1.0'] 
        value_5 = row['5.0']  
        value_6 = row['10.0'] 
        link = "http://mb.in.kyocera.co.jp/motionboard/main?mbid=fid5uk"

        message_text += f"超過アラート{TITLE}\n\n判定: {value_1}\n\n種別: {value_2}\n\n装置: {value_3}\n\n対象データ名: {value_4}\n\nデータ値: {value_5}\n\nしきい値: {value_6}\n\n↓MotionBoardへのリンク\n\n{link}\n\n----------------------------\n\n"

    payload = {"text": message_text}

    response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)

    if response.status_code == 200:
        logging.info("Step_4: Message sent successfully to Microsoft Teams")
        print("Step_4: Message sent successfully to Microsoft Teams")      
    else:
        logging.info(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
        print(f"Step_4: Failed to send message. Status code: {response.status_code}, Response: {response.text}")

send_teams_message(df)

In [ ]:
#5 
def update_json(df):
    config['insertion_time'] = str(df['timedate'].max())
    time=config['insertion_time']
    json_data = json.dumps(config, indent=2)

    with open(json_path, 'w') as output_file:
        output_file.write(json_data)

    logging.info(f"Step_5: Insertion Time is Updated: {time}")
    print(f"Step_5: Insertion Time is Updated {time}") 

update_json(df) 

In [ ]:
df = take_data_from_database() 
insert_into_database(df)
send_teams_message(df)
update_json(df)

In [ ]:
# 下記はテスト用のコード

In [ ]:
#Sending messages using for loop : OK

import requests
import json
import pandas as pd

data = {
    'Column_1': ['Title 1', 'Title 2', 'Title 3', 'Title 4', 'Title 5', 'Title 6', 'Title 7', 'Title 8', 'Title 9', 'Title 10'],
    'Column_2': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Column_3': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'Column_4': [21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
    'Column_5': [31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
    'Column_6': [41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
    'Column_7': [51, 52, 53, 54, 55, 56, 57, 58, 59, 60]
}

df = pd.DataFrame(data)

def send_teams_message_1(df):
    teams_webhook_url = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/c008d92acff64b93a35fa912edecdbf3/329f5970-aad5-4b39-90b5-cf6569c670e1"

    for index, row in df.iterrows():
        TITLE = row['Column_1']
        value_1 = row['Column_2']  
        value_2 = row['Column_3']  
        value_3 = row['Column_4']  
        value_4 = row['Column_5'] 
        value_5 = row['Column_6']  
        value_6 = row['Column_7'] 
        link = "http://mb.in.kyocera.co.jp/motionboard/main?mbid=fid5uk"

        message = f"{TITLE}\n\n判定: {value_1}\n\n種別: {value_2}\n\n装置: {value_3}\n\n対象データ名: {value_4}\n\nデータ値: {value_5}\n\nしきい値: {value_6}\n\n↓MotionBoardへのリンク\n{link}\n\n"

        payload = {"text": message}

        response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)

        if response.status_code == 200:
            print("Message sent successfully to Microsoft Teams!")
        else:
            print(f"Failed to send message. Status code: {response.status_code}, Response: {response.text}")

send_teams_message(df)

In [ ]:
# Send all rows in one excute  : OK
import requests
import json
import pandas as pd

data = {
    'Column_1': ['Title 1', 'Title 2', 'Title 3', 'Title 4', 'Title 5', 'Title 6', 'Title 7', 'Title 8', 'Title 9', 'Title 10'],
    'Column_2': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Column_3': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'Column_4': [21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
    'Column_5': [31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
    'Column_6': [41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
    'Column_7': [51, 52, 53, 54, 55, 56, 57, 58, 59, 60]
}

df = pd.DataFrame(data)

teams_webhook_url = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/c008d92acff64b93a35fa912edecdbf3/329f5970-aad5-4b39-90b5-cf6569c670e1"             

def send_teams_message_2(df):
    message_text = ""

    for index, row in df.iterrows():
        TITLE = row['Column_1']
        value_1 = row['Column_2']  
        value_2 = row['Column_3']  
        value_3 = row['Column_4']  
        value_4 = row['Column_5'] 
        value_5 = row['Column_6']  
        value_6 = row['Column_7'] 
        link = "http://mb.in.kyocera.co.jp/motionboard/main?mbid=fid5uk"

        message_text += f"{TITLE}\n\n判定: {value_1}\n\n種別: {value_2}\n\n装置: {value_3}\n\n対象データ名: {value_4}\n\nデータ値: {value_5}\n\nしきい値: {value_6}\n\n↓MotionBoardへのリンク\n\n{link}\n\n----------------------------\n\n"

    payload = {"text": message_text}

    response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)

    if response.status_code == 200:
        print("Message sent successfully to Microsoft Teams!")
    else:
        print(f"Failed to send message. Status code: {response.status_code}, Response: {response.text}")

send_teams_message_2(df)

In [ ]:
# DELETE

dsn = f"DATABASE={database_2};HOSTNAME={host_2};PORT={port_2};PROTOCOL={protocol_2};UID={user_2};PWD={password_2}"
conn = ibm_db.connect(dsn, "", "")
delete_query = "DELETE FROM THRESHOLD_ALERT"

try:
    ibm_db.autocommit(conn, ibm_db.SQL_AUTOCOMMIT_OFF)
    if ibm_db.exec_immediate(conn, delete_query):
        print("Existing data deleted successfully")

    ibm_db.commit(conn)

except Exception as e:
    print("Error:", e)
    ibm_db.rollback(conn)

In [ ]:
#2
def take_data_from_database():  
    try:
        logging.info("Step_2: Data is taken from the Database.")
        print("Step_2: Data is taken from the Database.")

        table = ['mm_01', 'mm_02', 'mm_03', 'mm_04']
        conn = psycopg2.connect(database=database_1, user=user_1, password=password_1, host=host_1, port=port_1)
        for table_name in table:
            sql_query = """
                        SELECT *
                        FROM %s
                        WHERE state IN (2, 3) AND timedate > %s;
                    """
            
            df = pd.read_sql_query(sql_query, conn, params=(table_name, inserted_time,)) 
            df['state'] = df['state'].replace({2: '警報', 3: '警告'})
      
            message_text = ""
            if df:
                for index, row in df.iterrows():
                    value_0 = row['timedate']
                    value_1 = row['state']  
                    value_2 = '気中パーティクル'
                    value_3 = table_name 

                    value_4 = row['0.5']
                    value_5 = row['1.0']  
                    value_6 = row['5.0'] 
                    value_7 = row['10.0']
                    
                    link = "http://mb.in.kyocera.co.jp/motionboard/main?mbid=fidkma77jxaifa65c65aaahv5mwqy"
                    message_text += f"超過アラート   : {value_0}\n\n判定: {value_1}\n\n種別: {value_2}\n\n装置: {value_3}\n\nデータ:\n\n\t 0.5  = {value_4}\n\t 1.0  = {value_5}\n\t 5.0  = {value_6}\n\t 10.0 = {value_7}\nしきい値: \n\n↓MotionBoardへのリンク\n\n{link}\n\n----------------------------\n\n"
        
                
                    payload = {"text": message_text}
                    response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)
                
                    if response.status_code == 200:
                        logging.info("Step_3: Message sent successfully to Microsoft Teams")
                        print("Step_3: Message sent successfully to Microsoft Teams")      
                    else:
                        logging.info(f"Step_3: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
                        print(f"Step_3: Failed to send message. Status code: {response.status_code}, Response: {response.text}")
                        return df
            else:
                logging.info(f"There is no state 2 or 3 data in {table_name}")
                print(f"There is no state 2 or 3 data in {table_name}")
                
    except Exception as e:
        logging.error(f"Step_2: Error while connecting to PostgreSQL: {e}")
        print(f"Step_2: Error while connecting to PostgreSQL: {e}")
        
            
send_teams_message(df)

In [ ]:
${CATEGORY} 
${EQP} 
${TS_DT}
${DATA_COLUMN} 
${INPUT_DT}
${INPUT_PERSON} 
${INPUT_COMMENT} 

import sys
import os
import logging
from logging.handlers import RotatingFileHandler
from datetime import datetime
from os.path import join


log = "logs"
os.makedirs(log, exist_ok=True)

log_path = join(log,"debug.log")
log_formatter = logging.Formatter('%(asctime)s [%(levelname)s]: %(message)s')
log_handler = RotatingFileHandler(filename=log_path, maxBytes=1048576, backupCount=10, delay=True)
log_handler.setFormatter(log_formatter)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)  
logger.addHandler(log_handler)


a0 = sys.argv[0]
a1 = sys.argv[1]
a2 = sys.argv[2]
a3 = sys.argv[3]
a4 = sys.argv[4]
a5 = sys.argv[5]
a6 = sys.argv[6]
a7 = sys.argv[7]

logging.info(f"0:{a0}\n1:{a1}\n2:{a2}\n3:{a3}\n4:{a4}\n5:{a5}\n6:{a6}\n7:{a7}")



In [ ]:
import os
from os.path import join
import logging

log = "logs"
os.makedirs(log, exist_ok=True)
log_path = join(log,"debug.log")



In [1]:
import sys
import pandas as pd 
import ibm_db
import sys
import os
import logging
from logging.handlers import RotatingFileHandler
from os.path import join


database = "AYB_APPL"
host = "10.143.16.244"
port ="50000"
protocol="TCPIP"
user="IOT_DATA"
password="asd23fgh"
log = "logs"


try:
    os.makedirs(log, exist_ok=True)
    logging.info("Log Folder is created")
except FileExistsError:
    logging.info("Failed to create Log Folder or Log Folder already exists")


log_path = join(log,"debug.log")
log_formatter = logging.Formatter('%(asctime)s [%(levelname)s]: %(message)s')
log_handler = RotatingFileHandler(filename=log_path, maxBytes=1048576, backupCount=10, delay=True)
log_handler.setFormatter(log_formatter)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)  
logger.addHandler(log_handler)

logging.info(f"Taking data to variable is started")

try:
    CATEGORY = sys.argv[1]
    EQP = sys.argv[2]
    TS_DT = sys.argv[3]
    DATA_COLUMN = sys.argv[4]
    INPUT_DT = sys.argv[5]
    INPUT_PERSON = sys.argv[6]
    INPUT_COMMENT = sys.argv[7]
    logging.info(f"\n1:{CATEGORY}\n2:{EQP}\n3:{TS_DT}\n4:{DATA_COLUMN}\n5:{INPUT_DT}\n6:{INPUT_PERSON}\n7:{INPUT_COMMENT}\n")

except Exception as e:
    logging.error(f"An error occurred: {str(e)}")



data = {
    "CATEGORY": [CATEGORY],
    "EQP": [EQP],
    "TS_DT": [TS_DT],
    "DATA_COLUMN": [DATA_COLUMN],
    "INPUT_DT": [INPUT_DT],
    "INPUT_PERSON": [INPUT_PERSON],
    "INPUT_COMMENT": [INPUT_COMMENT],
}

df = pd.DataFrame(data)

if df:
    logging.info("df is created")
    
def insert_into_database(df):
    logging.info("Inserting into database  is started")
    try:
        dsn = f"DATABASE={database};HOSTNAME={host};PORT={port};PROTOCOL={protocol};UID={user};PWD={password}"
        conn = ibm_db.connect(dsn, "", "")
        if conn:
            print("Connected to Database")
            logging.info("Connected to Database")
    except Exception as e:
        print("Failed to Connect to Database:", e)
        logging.info("Failed to Connect to Database")
        logging.info(f"Failed to Connect to Database:{e}")
    
    try:
        for index, row in df.iterrows():
            category = row["CATEGORY"]
            eqp = row["EQP"]
            ts_dt = row["TS_DT"]
            data_column = row["DATA_COLUMN"]
            input_dt = row["INPUT_DT"]
            input_person = row["INPUT_PERSON"]
            input_comment = row["INPUT_COMMENT"]

            sql = "UPDATE IOT_DATA.THRESHOLD_ALERT SET INPUT_DT=?, INPUT_PERSON=?, INPUT_COMMENT=? WHERE CATEGORY=? AND EQP=? AND TS_DT=? AND DATA_COLUMN=?"
            params = (input_dt, input_person, input_comment, category, eqp, ts_dt, data_column)

            stmt = ibm_db.prepare(conn, sql)
            if ibm_db.execute(stmt, params):
                print("Update successful")
                logging.info(f"Updated successful")
            else:
                print("Failed to update")
                logging.info(f"Failed to update")
    except Exception as e:
        print("Error occurred while updating:", e)
        logging.info("Error occurred while updating")
        logging.info(e)

insert_into_database(df)

INFO:root:Taking data to variable is started
ERROR:root:An error occurred: list index out of range


NameError: name 'EQP' is not defined

In [ ]:
import psycopg2
import pandas as pd 

database_1 =  "fms01"
host_1 = "10.143.64.54"
port_1 =  "5432"
protocol_1 = "TCPIP"
user_1 = "postgres"
password_1 = "fms"

conn = psycopg2.connect(database=database_1, user=user_1, password=password_1, host=host_1, port=port_1)

sql_query = """
    SELECT *
    FROM mm_01 
    WHERE state IN (2, 3);
"""

df = pd.read_sql_query(sql_query, conn)
df

In [ ]:
import ibm_db

database_2 = "AYB_APPL"
host_2 = "10.143.16.244"
port_2 ="50000"
protocol_2="TCPIP"
user_2="IOT_DATA"
password_2="asd23fgh"


dsn = f"DATABASE={database_2};HOSTNAME={host_2};PORT={port_2};PROTOCOL={protocol_2};UID={user_2};PWD={password_2}"
conn = ibm_db.connect(dsn, "", "")
if conn:
    print("Connected to the database")
else:
    print("Not connected to the database")


In [ ]:
insertion_time = dt.datetime.strptime("2023/10/20 23:59:55", '%Y/%m/%d %H:%M:%S')

import datetime as dt
import pandas as pd

time = dt.datetime.strptime("2023/10/20 23:59:55", '%Y/%m/%d %H:%M:%S')

sql_query = '''
    SELECT *
    FROM table
    WHERE state IN (2, 3) AND timestamp_column >= '{}';
'''.format(time)

df = pd.read_sql_query(sql_query, conn)

data = tuple(tuple(row) for row in df.values)
values = ",".join(map(str, data))

insert_query = f"INSERT INTO Table_Name (Column_1, Column_2, Column_3, Column_4) VALUES {values}"
stmt = ibm_db.prepare(conn, insert_query)

try:
    if ibm_db.execute(stmt, values):
        logging.info(f"{csv} : Inserted")
    with open(json_path, 'w') as output_file:
        output_file.write(json_data)
except:
    logging.info(f"{csv} : is already Updated")

import pandas as pd

data = {'Column_1': [1, 2, 3],
        'Column_2': ['A', 'B', 'C'],
        'Column_3': [4.5, 6.7, 8.1],
        'Column_4': ['X', 'Y', 'Z']}

df = pd.DataFrame(data, columns=['Column_1', 'Column_2', 'Column_3', 'Column_4'])

columns = df.columns
values = ",".join(["%s"] * len(columns))
data = [tuple(row) for row in df[columns].values]
insert_query = f"INSERT INTO Table_Name ({', '.join(columns)}) VALUES ({values})"

print("Sample DataFrame:")
print(df)
print("\nGenerated INSERT Query:")
print(insert_query)
data]



import pandas as pd

data = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
df = pd.DataFrame(data)

rows= tuple(tuple(row) for row in df.values)
string_rows = ",".join(map(str, rows))

import pandas as pd

data = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
df = pd.DataFrame(data)

rows= tuple(tuple(row) for row in df.values)
string_rows = ",".join(map(str, rows))

print(df)
print(df.va)
print(string_rows)

columns = ['Column_1', 'Column_2', 'Column_3', 'Column_4']
rows = [tuple(row) for row in df[columns].values]
values = ",".join(["%s"] * len(columns))
sql_query = f"INSERT INTO Table_Name ({', '.join(columns)}) VALUES ({values})"

rows = [tuple(row) for row in df[columns].values]

df[columns].values

data = tuple(tuple(row) for row in df.values)
values = ",".join(map(str, data))

with open(json_path) as json_file:
    config = json.load(json_file)

config['insertion_time'] = str(df['Timestamp'].max())
json_data = json.dumps(config, indent=2)

with open(json_path, 'w') as output_file:
    output_file.write(json_data)


    """
UPDATE IOT_DATA.THRESHOLD_ALERT 
SET INPUT_DT='2024/04/08 1:00:00', 
    INPUT_PERSON='Murodil', 
    INPUT_COMMENT='Test' 
WHERE CATEGORY='Particles' 
  AND EQP='MM_01' 
  AND TS_DT='2024/04/05 3:17:57' 
  AND DATA_COLUMN='0.5, 1.0, 5.0, 10.0'
"""


import ibm_db

database_2 = "AYB_APPL"
host_2 = "10.143.16.244"
port_2 ="50000"
protocol_2="TCPIP"
user_2="IOT_DATA"
password_2="asd23fgh"

dsn = f"DATABASE={database_2};HOSTNAME={host_2};PORT={port_2};PROTOCOL={protocol_2};UID={user_2};PWD={password_2}"
conn = ibm_db.connect(dsn, "", "")
if conn:
    print("Connected to the database")
else:
    print("Not connected to the database")